# Geophysical inversions with `SimPEG`
## An example with airborne electromagnetic data
Presenters:
Joseph Capriotti, Lindsey Heagy, Seogi Kang

In [ ]:
import pandas as pd
import numpy as np
from utilities.gex_parser import parse_gex_file
# If on google colab you will likely need to do instead:
# from gex_parse import parse_gex_file
import matplotlib.pyplot as plt

# Start with just importing the time domain module of simpeg, and a utility to plot a layered model.
from SimPEG.electromagnetics import time_domain as tdem
from SimPEG.utils import plot_1d_layer_model

import discretize

## Reading in the Data

First a bit about the skytem system. There are in general two sets of data:
* The low-moment data which is more sensitive to the near surface
* The high-moment data, which is (relatively) more sensitive to deeper structures.

The system has a hexagonal loop transmitter, with a $\frac{\partial \vec{B}}{\partial t}$ sensor.

There are two files that represent our data:
* A configuration file. Usually a `.gex` extension.
* The processed data file.
  * Simple CSV file $\rightarrow$ use `pandas`
 
We've provided a simple parser to read the information in the configuration file into python.

In [ ]:
# gex_file = 

In [ ]:
gex_file

Can simply use `pandas.read_csv` to read in the data file

In [ ]:
data_file = pd.read_csv('../data/MCWD3_dat.xyz')
# you can alternatively use pandas to directly fetch this file from the internet (if on colab)
# data_file = pd.read_csv("https://github.com/simpeg/segns-2024-tutorial/raw/main/data/MCWD3_dat.xyz")
data_file

inside this data file, missing data are marked with the value 9999, let's replace them with NaNs instead (which the plotting utilities will then ignore).

In [ ]:
# Plot the data file

Let's plot the location of all the stations in this survey

In [ ]:
plt.scatter(data_file.UTMX, data_file.UTMY, s=0.5)
plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)');

A few bits of processing have already been done to this data set, in particular, they have been normalized for source current strength, normalized by the transmitter area, and have also been spatially averaged along flight lines to form "stations".

Now we need a bit of information from the `.gex` file to tell us the number of high moment and low moment time gates for this survey

In [ ]:
# n_lm_gates = 
# n_hm_gates =

The data file will have `n_lm_gates` of db/dt data, `n_lm_gates` of relative error estimates, then `n_hm_gates` of the high moment db/dt data, followed by `n_hm_gates` relative error estimates.

### Zooming in to a single station to work with

Now let's look at a single station, along some randomly chosen line.

I can use panda's to easily group all of the data by a common line number
and then get that group

In [ ]:
# line_no
# line_grouping
# line

Let's look at all the data along that line.

In [ ]:
lm_data = line.iloc[:, 9:9+n_lm_gates]
hm_data = line.iloc[:, 9+2*n_lm_gates:9+2*n_lm_gates + n_hm_gates]

In [ ]:
# Plot the line data

In [ ]:
# select a single sounding along that line, and grab all of the data associated with that station
# Record 5590
# station = 

station_lm_data = station.iloc[0, 9:9+n_lm_gates].to_numpy()
station_lm_std = station.iloc[0, 9+n_lm_gates:9+2*n_lm_gates].to_numpy()
station_hm_data = station.iloc[0, 9+2*n_lm_gates:9+2*n_lm_gates + n_hm_gates].to_numpy()
station_hm_std = station.iloc[0,  9+2*n_lm_gates + n_hm_gates:].to_numpy()

Now we have numpy arrays of the recorded data for both the low and high moment data sets, along with their relative standard deviations.

In [ ]:
plt.scatter(data_file.UTMX, data_file.UTMY, s=0.5)
# add the line and station highlights

plt.xlabel('Easting (m)')
plt.ylabel('Northing (m)');

In [ ]:
plt.semilogy(line.UTMY, lm_data)
plt.semilogy(line.UTMY, hm_data)
# show where the station is along the line using axvline
plt.xlabel('Northing');

In order to model the data, we still need a few things pieces of information.
* The location in time of the measurements
* The transmitter waveforms
* The transmitter shape
* The receiver location

In [ ]:
# In general the location of the gates is
# gate_time = gate_centers + shift + delay
# The Gex file also tells us which gates the respective moments use
gate_centers = gex_file['General']['GateTimes']['center']

channel_info = gex_file['Channel1']
shift = channel_info['GateTimeShift']
delay = channel_info['MeaTimeDelay']
first_gate = channel_info['RemoveInitialGates']
last_gate = channel_info['NoGates']
# lm_times =

channel_info = gex_file['Channel2']
shift = channel_info['GateTimeShift']
delay = channel_info['MeaTimeDelay']
first_gate = channel_info['RemoveInitialGates']
last_gate = channel_info['NoGates']
# hm_times =

Some of the station's data are NaN's (which represent no data), Let's remove those points from our consideration

In [ ]:
lm_good_data = ~np.isnan(station_lm_data)
hm_good_data = ~np.isnan(station_hm_data)

lm_times = lm_times[lm_good_data]
hm_times = hm_times[hm_good_data]

station_lm_data = station_lm_data[lm_good_data]
station_lm_std = station_lm_std[lm_good_data]
station_hm_data = station_hm_data[hm_good_data]
station_hm_std = station_hm_std[hm_good_data]

# and of course reset our counts too
n_lm_gates = len(lm_times)
n_hm_gates = len(hm_times)

and finally plot some decay curves.

This is the data we will try to match.

In [ ]:
# loglog plots of time and data

### Setting up the survey

Now we want represent all of the configuration with `SimPEG` objects.

SimPEG's general structure to hold this information is:
* Create `Receiver`s (that observe data)
* Create `Source`s (and attach receivers to those sources)
* Gather all the sources into a `Survey`

In the time-domain EM module:

* `Receivers`
    * Observe at a specific location in space.
    * Observe a specific component (Here it will be the vertical component of $\frac{d\vec{B}}{dt}$)
    * At specific times (the time-gate centers).
 
* `Sources`
    * Have specific types (Line Current, Magnetic Dipole, etc.)
    * Have a location (or set of locations for Line current)
    * Have waveforms
    * Are "listened to" by receivers.

So let's start by grabbing the information we will need from the gex file.

We already have the time gate locations. So, let's grab all of the information related to the transmitter.

Specifically here, the waveforms of the low and high moment sources.

In [ ]:
waves = gex_file['General']['Waveforms']
# lm_wave_time
# lm_wave_form
# hm_wave_time
# hm_wave_form
# plot them in two subplots

The shape of the transmitter , and the offset of the receiver coil.

In [ ]:
# We pad the locations of the Tx points with 0, to expand it from a 2D (x,y) pair to a 3D (x, y, z) pair.
# We also add another row because we will need to close the transmitter loop.
# get the tx shape, pad it and close the loop
# tx_shape = 

# get the receiver offset
# rx_offset = 

# let's plot them

In [ ]:
# tx_shape cw or ccw?

Let's give the transmitter and receiver explicit locations by adding the UTM coordinates of the station to the transmitter shape and the receiver offset.

In [ ]:
# give them an absolute position (mostly just need to set the height here)
# tx_loc = tx_shape + ...
# rx_loc = rx_offset + ...
# also let's grab the area
# tx_area = 

We are often asked how to deal with both the low and high moment data in SimPEG. The answer is actually fairly straightforward as long as you understand the structure of a SimPEG survey, we model them as two seperate sources (who just so happen to be in the same location).

In [ ]:
# Low moment:
# rx_lm PointMagneticFluxTimeDerivative

# wave_lm PiecewiseLinearWaveform

# src_lm LineCurrent

In [ ]:
# high moment
# rx_hm

# wave_hm

# src_hm

Now we can set up a survey that has two sources!

In [ ]:
# srv

Now we're ready to set up the forward simulation, of which we will use (for this tutorial), a Layered simulation.

It has two physical properties:
* The conductivity of each layer (`sigma`)
* The thickness of each layer (`thicknesses`)

In [ ]:
#first a simple test
thicknesses = []
conductivities = [1E-2]

In [ ]:
sim = tdem.Simulation1DLayered(srv, sigma=conductivities, thicknesses=thicknesses)

We can set these properties on the simulation and create data. The only requirement is that the conductivity array must be one longer than the thickness array.

**note**: For this simulation the conductivities and thicknesses are defined from the surface downwards.

In [ ]:
# sim.thicknesses =
# sim.sigma = 
pre = sim.dpred(None)  # no "model" (will get to this more later).

pre_lm = pre[:n_lm_gates]
pre_hm = pre[n_lm_gates:]

plt.loglog(lm_times, -pre_lm)  # negative here to account for simpeg convention of z+ up
plt.loglog(hm_times, -pre_hm)
plt.loglog(lm_times, station_lm_data*tx_area, color='C0', marker='.', linestyle='')  # un-normalize by area
plt.loglog(hm_times, station_hm_data*tx_area, color='C1', marker='.', linestyle='')

# Setup an inversion

An inversion has many pieces that we must all setup for simpeg.

There are:
* `Map`: Objects that tell simpeg what to invert for.
* `Data`: a container for the observed data, its standard deviation, and the survey.
* `ObjectiveFunctions`, which describe the function we minimize to perform the inversion.
    * `DataMisfit`: How we measure the fitness of a model to the observed data.
    * `Regularization`: A measure of the simplicity of a model.
* `Minimization` Routines: Which method we use to iteratively minize the objective function.
* `InverseProblem`: Defines the optimization problem and the minimization routine.
* `Directives`: Operations that adjust parameters in the objective function as the inversion proceeds.
* `Inversion`: Groups together all of the above to actually run an inversion.

We will start by describing parametric inversions for simple 1D models, where the number of layers we invert for is much less than the number of data.

## Maps
This is how we tell simpeg what to invert for.

Maps define how we go from our inversion model to the physical properties.

In [ ]:
from SimPEG import maps

for example, this is a simple map that will transform its input according to:
$$out = e^{in}$$

In [ ]:
# exp_map =

This map is particularly useful for solving for conductivity for two reasons:
1) Generally conductivity varies on a logarithmic scale
2) We do not need to handle positivity constraints while minimizing

We use maps in `SimPEG` to tell simulations what we will be inverting for.

## Data
We need to create a container for the data, things to remember about here:

* The processed data were normalized by the transmitter area.
* There is a sign difference in the convention for SimPEG and the processed data
* The standard deviations are actually relative errors.

In [ ]:
from SimPEG import data

In [ ]:
# dobs
# rel_err

# data_container

Since the data container knows about the survey, we can directly index it
with sources and receivers to retrieve the data associated with that pair.

This is particularly useful to ensure that you have passed your observed data in the order that `SimPEG` expects it.

In [ ]:
data_container[src_hm, rx_hm]

## Parametric Halfspace inversion.

With a few key component concepts out of the way, Let's see how we can setup the layered simulation to solve for the best fitting halfspace.

Let's setup a new simulation that does this, we can re-use our previous survey object.

In [ ]:
sim_inv1 = tdem.Simulation1DLayered(srv, sigmaMap=exp_map)

When we assign a model (or pass a model to a function), the simulation uses the physical property maps to translate the model to the physical properties

In [ ]:
# try setting the model and looking at sigma

### Objective functions

SimPEG treats inversions as minimization functions, as such we need an objective function to minimize. For parametric inversions a data misfit is usually sufficient.

$$
\phi = \phi_d + \beta \phi_m
$$

We will start by defining just a simple data misfit function
* It needs to know how to compute data
* The data set you want to compare against.

It needs to be able to evaluate:
$$
\phi_d = |W_d ( d_{obs} - F(m))|^2
$$

In [ ]:
from SimPEG.data_misfit import L2DataMisfit

In [ ]:
# phi_d_1

We can evaluate the data misfit of a particular model by calling this object. (It also knows how to do derivative and (approximate) Hessian operations needed for minimization).

In [ ]:
# we can evaluate this function.

If we wanted to know the best fitting halfspace for this data we could simply minimze this function. Let's grab a Gauss Newton minimizer.

In [ ]:
from SimPEG import optimization

In [ ]:
minimizer = opt = optimization.InexactGaussNewton(
    maxIter=10, maxIterLS=20, maxIterCG=10, tolCG=1e-3
)
# Here, Inexact means we are going to use CG to solve for the step direction.

Define our inverse problem to be minimized:
One odd quirk of a SimPEG minimization is that we must pass a regularization object to our inverse problem. However we can set `beta=0` to ignore it.

In [ ]:
from SimPEG import regularization, inverse_problem

empty_reg = regularization.Smallness(discretize.TensorMesh([1]))
# note we needed to pass a mesh that had a single cell
# in it because our model has 1 value.

In [ ]:
#inv_prob_1

# sets up, phi_d + 0 * phi_m, meaning the minimizer is only going to act on the data misfit term.

In [ ]:
from SimPEG import inversion

In [ ]:
# inv1

In [ ]:
# Run inversion
recovered_model = inv1.run(m_0)

In [ ]:
# What was our recovered best fitting halfspace?

There's another useful way of computing data from a simulation, using the function
`make_synthetic_data` which returns a `Data` object instead of just the `numpy` array.

In [ ]:
# This function returns a data object
# (so we can easily index it with receivers for plotting)

In [ ]:
def plot_data(data_obj):
    plt.loglog(lm_times, -data_obj[src_lm, rx_lm])
    plt.loglog(hm_times, -data_obj[src_hm, rx_hm])
    
    plt.loglog(
        lm_times, -data_container[src_lm, rx_lm], color='C0', marker='x', linestyle=""
    )
    plt.loglog(
        hm_times, -data_container[src_hm, rx_hm], color='C1', marker='x', linestyle=""
    )
plot_data(data_pre)

clearly a 1D model fits this reasonably well (in general), can we do better?

### Multiple layers?

How do we solve for a multi (but few) layered model? We need to tell simpeg to invert for both conductivity and thickness.

Since we also need to ensure thicknesses are positive, lets use another `ExpMap` for them, but now our `model` consists of both values representing conductivity and thicknesses, so we need to tell SimPEG which parts correspond to each of them. There is a simple helper class to construct these called a `Wires`. It is very simple it says the first `X` values correspond to part 1, the next `Y` correspong to part 2.

In [ ]:
n_layers = 2
# wire_map

It contains to projection maps which split the model into its conductivities and thickness portions, we still need to use the `exp_map` to say we want to work in the log domain.

In [ ]:
# setup with exponent map
# sigma_map
# thick_map

In [ ]:
# let's use the best fitting half-space as our initial model
# setup initial model
# m_sigma_0_2
# m_h_0_2
m_0_2 = np.r_[m_sigma_0_2, m_h_0_2]

sim_inv2 = tdem.Simulation1DLayered(srv, sigmaMap=sigma_map, thicknessesMap=thick_map)

In [ ]:
# The simulation knows how to go from the model to the physical properties
sim_inv2.model = m_0_2
# sim_inv2. ...

Now we can set up the same pieces as the half-space inversion.

In [ ]:
phi_d_2 = L2DataMisfit(data=data_container, simulation=sim_inv2)

# create all of the same components of the inversion problem as before
minimizer2 = opt = optimization.InexactGaussNewton(
    maxIter=10, maxIterLS=20, maxIterCG=10, tolCG=1e-3
)

# again create a reg that has the same input size as the model vector, and then turn it off by setting `beta=0`
empty_reg = regularization.Smallness(discretize.TensorMesh([len(m_0_2)]))
inv_prob_2 = inverse_problem.BaseInvProblem(
    phi_d_2, reg=empty_reg, opt=minimizer, beta=0.0
)

And run it!

In [ ]:
inv2 = inversion.BaseInversion(inv_prob_2, [])

# Run inversion
recovered_model_2 = inv2.run(m_0_2)

In [ ]:
data_pre_2 = sim_inv2.make_synthetic_data(recovered_model_2)
sim_inv2.sigma, sim_inv2.thicknesses

In [ ]:
ax = plot_1d_layer_model(sim_inv2.thicknesses, sim_inv2.sigma)
ax.axvline(sim_inv1.sigma, color='C1')
ax.set_ylim([50, 0])
ax.set_xlim([1E-2, 1E0]);

In [ ]:
plot_data(data_pre_2)

We could keep going this route to add more and more layers, but at some point we start to enter the realm of underdetermined problems. E.g. when the number of model parameters exceed the number of observed data. For this we need to think about the regularization objective function.

## Regularized inversion

Let's keep building off of the pieces we had before. Let's discretize our model into many different layers. But this time we will only invert for the conductivity of each layer. For this we need to set up a mesh. This should be a 1D mesh, and it's single dimension would represent depth, aka we define z+ down for this example (Most simpeg is z+ up though).

We're going to define it with a single cell at 1m thickness, then increase the sizes geometrically by a factor of 1.05, for a total of 64 cells.

In [ ]:
# We can setup a mesh discretization using some shorthand:
# h = [dh, (dh, n_1), (dh, n_2, expansion_factor)
# h
# mesh

This time we only need to setup a simulation that inverts for conductivity, but we give it the thickness of each layer.
Which would be the `h[0]` property on the mesh, but truncated to 63 cells. (The last layer is interpreted as infinity depth.

In [ ]:
# sim_reg

Define yet another data misfit measure

In [ ]:
phi_d_reg = L2DataMisfit(data=data_container, simulation=sim_reg)

Now we need to also create a meaningful regularization function.

We're going to use something like:
$$
\phi_m = \alpha_v \int_V m + \alpha_x \int_v^C \frac{\partial m}{\partial x} dV 
$$

In [ ]:
# This class creates a function that measures the smallness
# and the smoothness of the model.
# reg WeightedLeastSquares

This class has some good default choices for $\alpha_s$ and $\alpha_x$

$$\alpha_s=1$$
$$\alpha_x = min(cell widths)$$

In [ ]:
# m_0_reg

In [ ]:

# create all of the same components of the inversion problem as before
minimizer_reg = opt = optimization.InexactGaussNewton(
    maxIter=10, maxIterLS=20, maxIterCG=10, tolCG=1e-3
)

# inv_prob_3

In [ ]:
inv3 = inversion.BaseInversion(inv_prob_3, [])

# Run inversion
recovered_model_3 = inv3.run(m_0_reg)

In [ ]:
data_pre_3 = sim_reg.make_synthetic_data(recovered_model_3)
ax = plot_1d_layer_model(sim_inv2.thicknesses, sim_inv2.sigma)
ax.axvline(sim_inv1.sigma, color='C1')
plot_1d_layer_model(sim_reg.thicknesses, sim_reg.sigma, ax=ax, color='C2')
ax.set_ylim([100, 0])
ax.set_xlim([1E-2, 1E0])

In [ ]:
plot_data(data_pre_3)

If we knew our noise level, We usually look for a model such that tit's $\phi_d(m) <= nD/2$. This comes from the expected value of a chi^2 disrtibutionl.

This process of manually adjusting the regularization parameter is a bit tiresome. In SimPEG we can use `Directive`s to automate this process for us.

Say we want to decrease the regularization parameter by a factor of 5 every two iterations:

In [ ]:
from SimPEG import directives

In [ ]:
# beta_cooler BetaSchedule

we also would likely want to keep track of the model and the function values at each iteration

In [ ]:
# save_dict SaveOutputDictEveryIteration

We combine these into a list, and pass them to the inversion.

In [ ]:
# create all of the same components of the inversion problem as before
# increase the number of iterations from 10 to 20, maybe do a few more CG iterations
minimizer_reg = opt = optimization.InexactGaussNewton(
    maxIter=10, maxIterLS=20, maxIterCG=10, tolCG=1e-3
)

# choose a starting beta
inv_prob_4 = inverse_problem.BaseInvProblem(
    phi_d_reg, reg=reg, opt=minimizer_reg, beta=10
)

inv4 = inversion.BaseInversion(inv_prob_4, [save_dict, beta_cooler])

# Run inversion
recovered_model_4 = inv4.run(m_0_reg)

The returned model here is just the last model from the inversion run, it might not necessarily be the best one though...

The `save_dict.outDict` dictionary is indexed per iteration, returning another dictionary containing the parameters and function evaluations. Let's collect all of the $\phi_d$, $\phi_m$ and $\beta$

In [ ]:
n_iter = save_dict.opt.iter
phi_ds = [save_dict.outDict[i]['phi_d'] for i in range(1, n_iter)]
phi_ms = [save_dict.outDict[i]['phi_m'] for i in range(1, n_iter)]
betas =  [save_dict.outDict[i]['beta'] for i in range(1, n_iter)]

We can make a few plots to investigate how these function evaluations change per iteration

In [ ]:
# Make some plots of phi_d, phi_m and beta
# choose a "final" model


And what does the model and data look like at this point?

In [ ]:
# m_final
d_final = sim_reg.make_synthetic_data(m_final)
ax = plot_1d_layer_model(sim_reg.thicknesses, exp_map * m_final)
plot_1d_layer_model(sim_inv2.thicknesses, sim_inv2.sigma, ax=ax)
ax.set_ylim([100, 0])
ax.set_xlim([1E-2, 1E0])

In [ ]:
plot_data(d_final)

**note** The overall fit looks fairly close.

You might ask..
"What is the noise level corresponding to this regularization parameter?"
There's a few ways you can define it, let's compare the relative sizes of the data residual vector to the observed data vector.

In [ ]:
# rel_diff